1\. Write a function that converts number representation (bin<->dec<->hex)

In [4]:
def conv(A):
    b = [A]
    a_bin = bin(A)
    b.append(a_bin)
    a_hex = hex(A)
    b.append(a_hex)
    return b
print (conv(41))

[41, '0b101001', '0x29']


In [9]:
def conv2(A):
    print('The number',A,'in binary representations is:',bin(A),'in hexadecimal representations is:',hex(A))
    return
conv2(41)

The number 41 in binary representations is: 0b101001 in hexadecimal representations is: 0x29


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [13]:
def convert_to_float(x):
    
    print(len(x))
    
    exp = 0
    mantissa = 1
    bias = 127
    s = x[0]
    
    for i in range(1,9):
        exp += int(x[i])*2**(8-i)
    print('exp =',exp)
        
    f = x[9:32]
    for i in range(len(f)):
        mantissa += int(f[i])*2**(-i-1)
    print('Mantissa =',mantissa)
    
    n = mantissa*2**(exp-bias)
    
    if int(s)==0:
        print('Il numero float è:',n)
    if int(s)==1:
        print('Il numero float è:',n)
    
    return

convert_to_float("00111111100000000000000000000000")

32
exp = 127
Mantissa = 1.0
Il numero float è: 1.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [26]:
import sys
print (sys.float_info)
import math

under=1.
over=1.

   
while (under != 0):
        vunder = under
        under = under / 2
while (over != math.inf):
        vover = over
        over *= 2
       
        
print('Overflow:',vover) 
print('Underflow:',vunder)


sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)
Overflow: 8.98846567431158e+307
Underflow: 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [38]:
x = 1
y = 2
eps = 1

while (x!=y):
    
    eps2 = eps
    y = x + eps
    eps /= 2
print(eps)




5.551115123125783e-17


In [37]:
original = 5
ma = 1
ori = 0
while (original != ori):
    ori = original
    original = original + eps/2
    eps = eps/2
print(eps)    

5.551115123125783e-17


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [20]:
import math
a = float(input())
b = float(input())
c = float(input())
def quadratic(a,b,c):
    x1 = (-b + math.sqrt(b*b - 4*a*c))/(2*a)
    x2 = (-b - math.sqrt(b*b - 4*a*c))/(2*a) 
    return (x1,x2)
def quadratic2(a,b,c):
    x_1 = (2*c)/(-b - math.sqrt(b*b - 4*a*c))
    x_2 = (2*c)/(-b + math.sqrt(b*b - 4*a*c))    
    return (x_1,x_2)
def quadratic3(a,b,c):
    x1 = (-b + math.sqrt(b*b - 4*a*c))/(2*a)
    x_2 = (2*c)/(-b + math.sqrt(b*b - 4*a*c))
    return (x1,x_2)
print('\n''a)''\n')
print(quadratic(a,b,c))
print('\n''b)''\n')
print(quadratic2(a,b,c))
print('\n''risultato più accurato per x2 ma non per x1.')
print('\n''c)''\n')
print(quadratic3(a,b,c))
print('\n''si utilizza la prima espressione per x1 mentre la seconda per x2.''\n')



0.001
1000
0.001

a)

(-9.999894245993346e-07, -999999.999999)

b)

(-1.000000000001e-06, -1000010.5755125057)

risultato più accurato per x2 ma non per x1.

c)

(-9.999894245993346e-07, -1000010.5755125057)

si utilizza la prima espressione per x1 mentre la seconda per x2.



6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [33]:
import math
delta = 0.01
x = 1

def function(x):
    f = x*(x-1)
    return f
def derivative(x,delta):
    d = (function(x+delta)-function(x))/delta
    return d
print('a)')
print(derivative(x,delta))
d = [pow(10,-2),pow(10,-4),pow(10,-6),pow(10,-8),pow(10,-10),pow(10,-12)]
print('b)')
for delta in d:
    print(derivative(x,delta))
    

a)
1.010000000000001
b)
1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [4]:
import numpy as np
import time
N = 100

def yk(x):
    y = np.sqrt(1-x**2)
    return y 
def integral(N):
    h = 2/N
    I = 0
    for k in range(N):
        I += h*yk(k/N)        
    return I
print('a)\nrisultato integrale:',integral(100))
print('risultato reale:',np.pi/2)
print('differenza:',integral(100)-np.pi/2,'\nb)')
#%timeit integral(100000)
#start_time = time.process_time()
#un minuto
#print("The result for one minute is",integral(40000000))
#un secondo
#print("The result for one second is",integral(270000))
#print("--- %s seconds ---" % (time.process_time() - start_time))
m = integral(40000000)
s = integral(270000)
diff = (s-m)
print('In un minuto si ha:',m,'in un secondo:',s,'il guadagno è:',diff)

a)
risultato integrale: 1.5802085158895236
risultato reale: 1.5707963267948966
differenza: 0.009412189094627044 
b)
In un minuto si ha: 1.5707963517924732 in un secondo: 1.5708000263075346 il guadagno è: 3.674515061335981e-06
